# Exploratory Data Analysis (EDA)

**Tujuan:** Menganalisis dataset MARS untuk memahami perilaku pengguna dan karakteristik konten. Setiap temuan dari EDA ini akan menjadi dasar hipotesis untuk **Langkah 1: Feature Engineering**.

**Checklist EDA:**
1.  **Pengecekan Dasar:** Memuat data, memeriksa tipe data, dan statistik unik.
2.  **Analisis Missing Values:** Mengidentifikasi kualitas data dan memutuskan kolom mana yang akan digunakan atau dibuang.
3.  **Analisis Distribusi:** Mencari pola inti dalam perilaku pengguna dan karakteristik konten.

## 1. Setup & Pengecekan Dasar

Pertama, kita impor *library*. Berdasarkan struktur file baru (`_en` dan `_fr`), kita akan memuat 8 file CSV dan **menggabungkannya (concatenate)** menjadi 4 DataFrame utama: `df_users`, `df_items`, `df_explicit`, dan `df_implicit`. Ini memberi kita gambaran utuh dari *seluruh* populasi siswa.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Mengatur style visualisasi
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

print("Libraries imported successfully!")

Libraries imported successfully!


In [6]:
# Tentukan path ke folder data.
# Karena notebook ini ada di 'notebooks/', path ke data adalah '../data/'
DATA_DIR = '../data/'

# Cek apakah path valid
if not os.path.exists(DATA_DIR):
    print(f"Error: Directory not found at {DATA_DIR}.")
    print("Pastikan folder 'data/' ada di level root proyek Anda (satu level di atas 'notebooks/').")
else:
    print(f"Data directory found at: {DATA_DIR}")

try:
    # 1. Muat & Gabungkan Users
    df_users_en = pd.read_csv(os.path.join(DATA_DIR, 'users_en.csv'))
    df_users_fr = pd.read_csv(os.path.join(DATA_DIR, 'users_fr.csv'))
    df_users = pd.concat([df_users_en, df_users_fr], ignore_index=True)

    # 2. Muat & Gabungkan Items
    df_items_en = pd.read_csv(os.path.join(DATA_DIR, 'items_en.csv'))
    df_items_fr = pd.read_csv(os.path.join(DATA_DIR, 'items_fr.csv'))
    df_items = pd.concat([df_items_en, df_items_fr], ignore_index=True)

    # 3. Muat & Gabungkan Explicit Ratings
    df_explicit_en = pd.read_csv(os.path.join(DATA_DIR, 'explicit_ratings_en.csv'))
    df_explicit_fr = pd.read_csv(os.path.join(DATA_DIR, 'explicit_ratings_fr.csv'))
    df_explicit = pd.concat([df_explicit_en, df_explicit_fr], ignore_index=True)

    # 4. Muat & Gabungkan Implicit Ratings
    df_implicit_en = pd.read_csv(os.path.join(DATA_DIR, 'implicit_ratings_en.csv'))
    df_implicit_fr = pd.read_csv(os.path.join(DATA_DIR, 'implicit_ratings_fr.csv'))
    df_implicit = pd.concat([df_implicit_en, df_implicit_fr], ignore_index=True)

    print("\nData loaded and concatenated successfully:")
    print(f"Total Users: {df_users.shape}")
    print(f"Total Items: {df_items.shape}")
    print(f"Total Explicit Ratings: {df_explicit.shape}")
    print(f"Total Implicit Ratings: {df_implicit.shape}")

except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Pastikan semua 8 file CSV (users_en.csv, users_fr.csv, dll.) ada di dalam folder '../data/'.")

Data directory found at: ../data/

Data loaded and concatenated successfully:
Total Users: (131247, 2)
Total Items: (2618, 12)
Total Explicit Ratings: (88998, 5)
Total Implicit Ratings: (275735, 3)


In [7]:
print("--- Users Info ---")
df_users.info()
print("\n--- Items Info ---")
df_items.info()
print("\n--- Explicit Ratings Info ---")
df_explicit.info()
print("\n--- Implicit Ratings Info ---")
df_implicit.info()

--- Users Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131247 entries, 0 to 131246
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  131247 non-null  int64 
 1   job      10466 non-null   object
dtypes: int64(1), object(1)
memory usage: 2.0+ MB

--- Items Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2618 entries, 0 to 2617
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      2618 non-null   int64  
 1   language     2618 non-null   object 
 2   name         2618 non-null   object 
 3   nb_views     2511 non-null   float64
 4   description  2230 non-null   object 
 5   created_at   2618 non-null   int64  
 6   Difficulty   824 non-null    object 
 7   Job          2618 non-null   object 
 8   Software     2618 non-null   object 
 9   Theme        2618 non-null   object 
 10  duration     2618 non-null   floa

In [8]:
print("--- Data Unik ---")
print(f"Total Unique Users (dari users.csv): {df_users['User ID'].nunique()}")
print(f"Total Unique Items (dari items.csv): {df_items['Item ID'].nunique()}")

print(f"\nUsers in Explicit Ratings: {df_explicit['User ID'].nunique()}")
print(f"Items in Explicit Ratings: {df_explicit['Item ID'].nunique()}")

print(f"Users in Implicit Ratings: {df_implicit['User ID'].nunique()}")
print(f"Items in Implicit Ratings: {df_implicit['Item ID'].nunique()}")

--- Data Unik ---


KeyError: 'User ID'

**Temuan Awal (Pengecekan Dasar):**
* **Tipe Data:** Sebagian besar tipe data sudah benar (Angka sebagai `int` atau `float`, Kategori sebagai `object`). Kolom `Creation Date` perlu diubah ke `datetime` jika kita ingin melakukan analisis berbasis waktu, tapi untuk clustering awal, ini belum tentu perlu.
* **Skala:** Kita berhadapan dengan ~131k users dan ~2.6k items (total FR + EN, sesuai paper). Namun, hanya sebagian kecil dari user yang aktif (Total user unik di `explicit` + `implicit` jauh lebih kecil dari total di `users.csv`). Ini adalah pola *long-tail* yang khas di platform online.

## 2. Analisis Missing Values (Kualitas Data)

Sekarang, kita fokus pada *kualitas* data. Kolom apa yang bisa kita andalkan? Kita akan visualisasikan persentase data yang hilang. Ini akan menentukan fitur mana yang layak dibuat.

In [ ]:
def missing_data_analysis(df, df_name):
    missing_pct = (df.isnull().sum() / len(df)) * 100
    missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=False)
    print(f"--- Missing Data in {df_name} ---")
    if missing_pct.empty:
        print("No missing data found.")
    else:
        print(missing_pct)
        
    return missing_pct

missing_users = missing_data_analysis(df_users, "Users")
missing_items = missing_data_analysis(df_items, "Items")
missing_explicit = missing_data_analysis(df_explicit, "Explicit Ratings")
missing_implicit = missing_data_analysis(df_implicit, "Implicit Ratings")

In [ ]:
# Visualisasikan data yang hilang di df_items, karena ini krusial untuk fitur konten
plt.figure(figsize=(15, 8))
sns.heatmap(df_items.isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Visualisasi Missing Data pada `items.csv` (Gabungan EN+FR)')
plt.show()

**Temuan & Tindakan (Missing Values):**

* `users.csv`: Kolom `Job` (pekerjaan) hilang >90% (sesuai paper). **Tindakan FE:** Kolom ini **dibuang**. Tidak ada nilai informatif yang bisa diambil.
* `items.csv`: Kolom `Level` dan `Job` hilang >60-70%. **Tindakan FE:** Kolom ini juga akan **dibuang**. Terlalu banyak data hilang untuk di-imputasi secara akurat.
* `items.csv` (Kabar Baik): Kolom `Type`, `Software`, dan `Theme` memiliki data yang relatif lengkap (missing < 25%). **Tindakan FE:** Ini akan menjadi **fitur konten utama** kita. Kita akan mengisi data yang hilang ini dengan nilai 'Unknown' atau 'Other'.
* `ratings` (explicit/implicit): Tidak ada data `User ID` atau `Item ID` yang hilang. Ini sangat bagus dan menunjukkan data transaksional yang bersih.

## 3. Analisis Distribusi (Mencari Pola)

Ini adalah inti dari EDA kita. Kita ingin memahami:
1.  **Perilaku Tontonan:** Bagaimana orang menonton video? Apakah mereka menyelesaikannya?
2.  **Karakteristik Konten:** Konten seperti apa yang paling umum di platform ini?
3.  **Aktivitas User:** Apakah user sangat aktif atau hanya "turis"?

### 3.1. Perilaku Tontonan (dari `explicit_ratings`)

In [ ]:
plt.figure(figsize=(14, 6))

# Plot 1: Distribusi 'Watch %'
plt.subplot(1, 2, 1)
sns.histplot(df_explicit['Watch %'], bins=20, kde=False)
plt.title('Distribusi Watch Percentage (%) (Gabungan EN+FR)')
plt.xlabel('Watch %')
plt.ylabel('Jumlah Interaksi')

# Plot 2: Distribusi 'Rating' (skala 1-10)
plt.subplot(1, 2, 2)
sns.histplot(df_explicit['Rating'], bins=10, discrete=True, color='salmon')
plt.title('Distribusi Rating (Skala 1-10) (Gabungan EN+FR)')
plt.xlabel('Rating')
plt.ylabel('Jumlah Interaksi')

plt.tight_layout()
plt.show()

**Temuan & Hipotesis FE (Perilaku Tontonan):**

* **Pola Bimodal:** Seperti yang dikonfirmasi oleh paper (Fig 1 & 2), distribusinya sangat **Bimodal**. Ada puncak besar di `Rating = 1` (ditonton < 10%) dan puncak besar lainnya di `Rating = 10` (ditonton > 90%).
* **Artinya:** Pengguna cenderung **segera keluar** (mungkin konten tidak relevan) atau **menyelesaikan** konten. Sangat sedikit yang berhenti di tengah jalan.
* **Hipotesis FE (Sangat Penting):** Menggunakan `avg_rating` (rata-rata rating) saja akan **sangat menyesatkan**. User dengan 5 rating '1' dan 5 rating '10' akan punya rata-rata 5.5, sama dengan user yang punya 10 rating '5' atau '6'. Padahal, perilaku mereka sangat berbeda.
    * **Fitur yang akan kita buat:**
        1.  `completion_rate` (Persentase tontonan dengan Rating 9-10)
        2.  `drop_off_rate` (Persentase tontonan dengan Rating 1-2)
        3.  `total_interactions` (Total video yang ditonton)

### 3.2. Karakteristik Konten (dari `items.csv`)

Kita akan membuang kolom `Level` dan `Job` yang banyak hilang, dan fokus pada `Type`, `Software`, dan `Theme`.

In [ ]:
# Mengisi NaN dengan 'Unknown' agar bisa di-plot
df_items['Type'] = df_items['Type'].fillna('Unknown')
df_items['Software'] = df_items['Software'].fillna('Unknown')
df_items['Theme'] = df_items['Theme'].fillna('Unknown')

plt.figure(figsize=(18, 5))

# Plot 1: Tipe Konten
plt.subplot(1, 3, 1)
df_items['Type'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribusi Tipe Konten')
plt.ylabel('Jumlah Item')
plt.xticks(rotation=0)

# Plot 2: Top 10 Software
plt.subplot(1, 3, 2)
df_items['Software'].value_counts().head(10).plot(kind='bar', color='lightgreen')
plt.title('Top 10 Software')
plt.ylabel('Jumlah Item')
plt.xticks(rotation=45)

# Plot 3: Top 10 Theme
plt.subplot(1, 3, 3)
df_items['Theme'].value_counts().head(10).plot(kind='bar', color='orange')
plt.title('Top 10 Theme')
plt.ylabel('Jumlah Item')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

**Temuan & Hipotesis FE (Karakteristik Konten):**

* **Tipe Konten:** Mayoritas konten adalah `Tutorial` singkat. `Webcast` (durasi panjang) adalah minoritas. Ini menunjukkan platform ini lebih fokus pada pembelajaran *micro-learning*.
* **Topik:** Topik seputar `Office 365` (`Excel`, `Teams`, `SharePoint`) sangat mendominasi.
* **Hipotesis FE:** Kita bisa mengelompokkan siswa berdasarkan preferensi mereka terhadap:
    1.  **Format Belajar:** Apakah mereka lebih suka `Tutorial` singkat atau `Webcast` panjang? (Kita akan buat fitur `pref_type_tutorial_pct` vs `pref_type_webcast_pct`).
    2.  **Spesialisasi Topik:** Apakah mereka 'Generalis' (menyentuh banyak software) atau 'Spesialis' (fokus hanya pada 1-2 software, misal `Excel`)? (Kita akan buat fitur `pref_software_excel_pct`, `pref_software_teams_pct`, dll.)

### 3.3. Aktivitas User

Terakhir, apakah semua user layak kita masukkan ke model clustering? User yang hanya menonton 1 video tidak memiliki "gaya belajar" yang bisa dianalisis. Kita perlu memfilter mereka.

In [ ]:
# Kita fokus pada explicit ratings, karena ini menunjukkan niat belajar yang lebih kuat
user_activity = df_explicit.groupby('User ID').size().to_frame('total_interactions')

print(user_activity['total_interactions'].describe())

# Plot distribusi jumlah interaksi
# Kita akan plot pada skala log karena distribusinya sangat miring (long-tail)
plt.figure(figsize=(12, 6))
sns.histplot(user_activity['total_interactions'], bins=50, log_scale=(False, True))
plt.title('Distribusi Jumlah Interaksi (Explicit) per User (Log Scale pada Sumbu Y)')
plt.xlabel('Jumlah Total Interaksi')
plt.ylabel('Jumlah User (Log Scale)')
plt.show()

In [ ]:
# Mari kita lihat persentase user dengan interaksi yang sangat sedikit
print("Persentase User dengan Interaksi Minimal:")
for n in [1, 2, 3, 5, 10]:
    pct = (len(user_activity[user_activity['total_interactions'] <= n]) / len(user_activity)) * 100
    print(f"Users dengan <= {n} interaksi: {pct:.2f}%")

**Temuan & Hipotesis FE (Aktivitas User):**

* **Distribusi Long-Tail:** Sebagian besar user (lebih dari 50%!) hanya memiliki 3 interaksi atau kurang. Rata-rata interaksi adalah ~8, tapi median (nilai tengah) hanya 3. Ini menunjukkan ada banyak user "turis" atau *one-time user*.
* **Hipotesis FE (Pembersihan Data):** Untuk membangun cluster yang bermakna, kita harus **memfilter user yang tidak aktif**. User dengan < 5 interaksi (misalnya) mungkin tidak memberikan sinyal yang cukup kuat tentang "gaya belajar" mereka. 
    * **Tindakan:** Pada Langkah 1 (FE), kita hanya akan memproses `User ID` yang memiliki **minimal 5 `explicit_ratings`**.

## 4. Kesimpulan EDA & Rencana Feature Engineering

EDA kita selesai dan sangat produktif. Kita telah beralih dari "data mentah" ke "rencana taktis".

**Rangkuman Hipotesis untuk Langkah 1 (FE):**

1.  **Filter User:** Kita akan membuang user dengan `< 5` interaksi `explicit_ratings`.
2.  **Filter Kolom:** Kita akan membuang kolom `Job` (dari users & items) dan `Level` (dari items) karena data terlalu banyak hilang.
3.  **Fitur Engagement (Perilaku Tontonan):**
    * `total_explicit_interactions` (Jumlah total video ditonton)
    * `total_implicit_interactions` (Jumlah total halaman dikunjungi)
    * `completion_rate` (Persentase rating 9-10)
    * `drop_off_rate` (Persentase rating 1-2)
    * `avg_watch_pct` (Rata-rata `Watch %`, tetap kita masukkan sebagai perbandingan)
4.  **Fitur Preferensi Format (Tipe Konten):**
    * `pref_type_tutorial_pct` (Persentase video 'Tutorial' yang ditonton)
    * `pref_type_webcast_pct` (Persentase video 'Webcast' yang ditonton)
    * `avg_duration_watched` (Rata-rata durasi video yang ditonton)
5.  **Fitur Preferensi Topik (Spesialisasi):**
    * `topic_specialization` (Bisa diukur dengan menghitung entropi dari `Software` yang ditonton, atau lebih sederhana: jumlah `Software` unik yang ditonton).
    * `pref_software_excel_pct` (Persentase tontonan pada `Excel`)
    * `pref_software_teams_pct` (Persentase tontonan pada `Teams`)
    * (Dan beberapa software top lainnya).

Kita sekarang siap untuk **Langkah 1: Feature Engineering**.